In [ ]:
import pandas as pd
import numpy as np
import os
import re
import json

In [ ]:
from google.colab import files
f = files.upload()

In [ ]:
os.listdir()

In [ ]:
d1 = pd.read_csv('/content/Coupang_products1.csv')
d2 = pd.read_csv('/content/Coupang_products2.csv')
display(d1.head())
display(d2.head())

In [ ]:
d1.shape, d2.shape

In [ ]:
d1['Product_Name'] = d1['Product_Name'].str.replace('\n\n\n',' ').str.replace('★', ' ')
d2['Product_Name'] = d2['Product_Name'].str.replace('\n\n\n',' ').str.replace('★', ' ')

In [ ]:
def remove_mandarin(df):
  # 한자 제거 정규표현식
  regex_pattern = re.compile(r'[\u4e00-\u9fff]')
  # 특수 문자 제거 정규표현식
  regex_pattern2 = re.compile(r'[^a-zA-Z0-9가-힣\s]')  # 영문자, 숫자, 한글, 공백을 제외한 모든 문자 매치
  lst = df.tolist()
  lst_after = []

  for l in lst:
    result_string = regex_pattern.sub('', l)
    result_string2 = regex_pattern.sub('', result_string)
    lst_after.append(result_string2)

  return lst_after

In [ ]:
d1['Product_Name'] = remove_mandarin(d1['Product_Name'])
d2['Product_Name'] = remove_mandarin(d2['Product_Name'])

In [ ]:
str(d1['Price'])

In [ ]:
all_data = pd.concat([d1,d2]).reset_index(drop=True)
all_data['text'] = all_data['Price'].astype('string') + ' ' + all_data['Product_Name']

In [ ]:
df = all_data['text'].str.replace('\n\n\n',' ')
df.head()

In [ ]:
df.to_csv('all_data_coupang.txt', index=False, header=False, sep='\n')

In [ ]:
df.to_csv('all_data_coupang.csv', encoding = 'CP949', index= False)

### json 파일 전처리

In [ ]:
with open("/content/Naver_shopping1.json", "r", encoding="utf-8") as json_file:
    train = json.load(json_file)
with open("/content/Naver_shopping2.json", "r", encoding="utf-8") as json_file:
    train2 = json.load(json_file)

In [ ]:
json_lst = []
for t1 in train:
  json_lst.append(t1['title'] + ' ' + str(t1['lprice']))
for t2 in train2:
  json_lst.append(t2['title'] + ' ' + str(t2['lprice']))

In [ ]:
json_df = pd.DataFrame({'text':json_lst})
json_df.head()

In [ ]:
json_df.to_csv('naver_data.csv', encoding = 'utf-8', index= False)

In [ ]:
# 텍스트 파일 경로
txt_file_path = '/content/all_data_coupang.txt'

# 텍스트 파일 읽기
with open(txt_file_path, 'r', encoding='utf-8') as file:
    text_content = file.read()

# 읽어온 텍스트 출력
print(text_content)

In [ ]:
def create_labeled_data(line):
    # Assuming the first 5 characters are the price and the rest is the product name
    price = line[:5]
    product_name_start = 6  # Product name starts after the price
    product_name_end = line.find(',', product_name_start)  # Finding the end of the product name
    if product_name_end == -1:
        product_name_end = len(line)  # If no comma, the product name goes till the end of the line

    return {
        "sentence": line.strip(),
        "labels": [
            {"start": 0, "end": 4, "entity": "가격"},
            {"start": product_name_start, "end": product_name_end, "entity": "상품명"}
        ]
    }

# Reading the text file
file_path = '/content/all_data_coupang.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Creating labeled data for all lines
labeled_data = [create_labeled_data(line) for line in lines if line.strip()]

# Convert the labeled data to JSON format
labeled_json = json.dumps({'data': labeled_data}, ensure_ascii=False, indent=2)

# Writing the JSON to a file
output_file_path = '/content/labeled_data.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(labeled_json)
